In [1]:
from sys import path
path.insert(0,'../python')
import numpy as np
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import unicodedata
import json
from googlemaps import Client
from yaml import load as yaml_load, FullLoader
from google_api import *
import folium
from folium.plugins import MarkerCluster,HeatMap, Fullscreen
from async_request import make_request

# Obtenemos la lista de pueblos mágicos

In [2]:
url = 'https://www.gob.mx/sectur/articulos/pueblos-magicos-206528'

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = get(url, headers= headers)
response.status_code

200

In [4]:
content = unicodedata.normalize("NFKD", response.content.decode('utf-8'))
soup = BeautifulSoup(content, 'html.parser')
#print(soup.prettify())

In [5]:
ol = soup.find_all('ol')[1]
pueblos_text = [li.get_text() for li in ol.find_all('li')]

In [7]:
pueblos = []
for i, pueblo in enumerate(pueblos_text):
    pueblos.append({'name':pueblo})

# with open('../data/pueblos_magicos.json', 'w') as outfile:
#     json.dump(pueblos, outfile)
len(pueblos)

121

# Obtención de las coordenadas de los pueblos mágicos

In [11]:
yaml_file = yaml_load(open('../auth/google.yaml'), Loader=FullLoader)
GOOGLE_API_KEY = yaml_file['GOOGLE_API_KEY']
gmaps = Client(key=GOOGLE_API_KEY, queries_per_second = 500)

In [12]:
with open('../data/pueblos_magicos.json') as json_file:
    pueblos = json.load(json_file)

### Opción 1

```python
coordenadas = []
for pueblo in pueblos:
    geocode_result = gmaps.geocode(pueblo['name'] + ' Mexico')
    coords = get_geocode_locations(geocode_result)[0]
    coordenadas.append(coords)
    
```

### Opción 2

```python
from async_request import make_request

def fetch_geocode(session, data, i):
        try:
            response = gmaps.geocode(data)
            return response
        except Exception as e:
            print('Error trying to request the directions', e)
            return []
        
data = [pueblo['name'] + ', Mexico' for pueblo in pueblos]

geocode_results = make_request(fetch_geocode, data) 

coordinates = [get_geocode_locations(geocode_result)[0] for geocode_result in geocode_results]

for i,pueblo in enumerate(pueblos):
    pueblo['coords'] = coordinates[i]
    
```

In [14]:
# with open('../data/pueblos_magicos.json', 'w') as outfile:
#     json.dump(pueblos, outfile)

## Visualización de los pueblos en el mapa

In [15]:
with open('../data/pueblos_magicos.json') as json_file:
    pueblos = json.load(json_file)

In [16]:
tiles_url = 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'
attrb ='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
m = folium.Map(tiles=None)

folium.TileLayer(tiles_url,attr=attrb,name = 'Pueblos Mágicos de México', show = True).add_to(m)

text = '<h4>Planta Bosch Aguascalientes</h4>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)


pt_lyr = folium.FeatureGroup(name = 'Pueblos Mágicos',show=True)

for pueblo in pueblos:
    point = pueblo['coords']
    name = pueblo['name']
    text = f'<h4>{name}</h4>'
    text = folium.Html(text, script=True)
    popup = folium.Popup(text, max_width=2650)

    folium.CircleMarker(point,radius = 4,
                    popup = popup,
                    fill=True, # Set fill to True|
                    fill_color='white',
                    color = 'green',
                    fill_opacity=1).add_to(pt_lyr)
pt_lyr.add_to(m)

Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds(m.get_bounds())
m
#m.save('../maps/Mapeo Asociados Aguascalientes.html')

## Obtención de la matriz de distancia y tiempo

In [17]:
coordinates = [pueblo['coords'] for pueblo in pueblos]

In [18]:
origins = [coordinates[1]]
center = len(coordinates) // 2
destinations1 = coordinates[:center]
destinations2 = coordinates[center:]

In [19]:
distance_matrix_result1 = gmaps.distance_matrix(origins, destinations1, avoid = None) #avoid = 'ferries'
distance_matrix_result2 = gmaps.distance_matrix(origins, destinations2, avoid = None) #avoid = 'ferries'

In [20]:
distances = get_distance_matrix(distance_matrix_result1)[0] + get_distance_matrix(distance_matrix_result2)[0]
durations = distances = get_distance_matrix(distance_matrix_result1)[0] + get_distance_matrix(distance_matrix_result2)[0]

In [21]:
def fetch_distance_matrix(session, data, i):
        origins, destinations = data

        try:
            center = len(destinations) // 2
            destinations1 = destinations[:center]
            destinations2 = destinations[center:]
            distance_matrix_result1 = gmaps.distance_matrix(origins, 
                                                            destinations1, 
                                                            avoid = None) #avoid = 'ferries'
            distance_matrix_result2 = gmaps.distance_matrix(origins, 
                                                            destinations2, 
                                                            avoid = None) #avoid = 'ferries'
            return [distance_matrix_result1, distance_matrix_result2]
        except Exception as e:
            print('Error trying to request the directions', e)
            return []

```python
destinations = coordinates
data = ([[coord], destinations] for coord in coordinates)
distance_matrix_results = make_request(fetch_distance_matrix, data) 
```

```python
distance_matrix = []
duration_matrix = []
for distance_matrix_result1, distance_matrix_result2 in distance_matrix_results:
    distances = get_distance_matrix(distance_matrix_result1)[0] + get_distance_matrix(distance_matrix_result2)[0]
    durations = get_duration_matrix(distance_matrix_result1)[0] + get_duration_matrix(distance_matrix_result2)[0]
    distance_matrix.append(distances)
    duration_matrix.append(durations)
    
    
distance_matrix_df = pd.DataFrame(distance_matrix)
duration_matrix_df = pd.DataFrame(duration_matrix)

distance_matrix_df.to_csv('../data/distance_matrix.csv')
duration_matrix_df.to_csv('../data/duration_matrix.csv')

```

In [51]:
distance_matrix = pd.read_csv('../data/distance_matrix.csv', index_col=0)
duration_matrix = pd.read_csv('../data/duration_matrix.csv', index_col=0)

In [ ]:
dista

In [31]:
result = gmaps.directions('Isla Mujeres Mexico', 'Cancun, Mexico',  avoid =  None)

In [32]:
polyline = get_directions_polylines(result)[0]

In [33]:
tiles_url = 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'
attrb ='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
m = folium.Map(tiles=None)

folium.TileLayer(tiles_url,attr=attrb,name = 'Pueblos Mágicos de México', show = True).add_to(m)

text = '<h4>Planta Bosch Aguascalientes</h4>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)


pt_lyr = folium.FeatureGroup(name = 'Pueblos Mágicos',show=True)


folium.PolyLine(decode_polyline(polyline)).add_to(pt_lyr)
pt_lyr.add_to(m)

Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds(m.get_bounds())
m
#m.save('../maps/Mapeo Asociados Aguascalientes.html')